In [2]:
import paddle
import paddle.nn.functional as F
import numpy as np
import pandas as pd
import os
from paddle import nn
from PIL import Image

IMAGE_SIZE = 224

print(paddle.__version__)

2.4.1


In [3]:
# 定义数据集
class MyDataset(paddle.io.Dataset):
    def __init__(self, img_dir='data/PALM-Training400/', csv_dir='data/Classification.csv') -> None:
        super(MyDataset, self).__init__()
        self.csvfile = pd.read_csv(csv_dir)
        self.imgpath = img_dir
        pass
    def __len__(self):
        return len(self.csvfile)
    def __getitem__(self, idx):
        img = np.reshape((np.array(Image.open(self.imgpath+os.sep+self.csvfile['imgName'][idx]).resize((IMAGE_SIZE,IMAGE_SIZE))).astype('float32')),(3,IMAGE_SIZE,IMAGE_SIZE))/256.
        lab = np.array(self.csvfile['Label'][idx])
        return img,lab
    pass
mydataset = MyDataset()
mydataloader = paddle.io.DataLoader(
    mydataset,
    batch_size=4
)

In [4]:
# 定义网络结构
def vgg_block(num_convs, in_channels, out_channels):
    net = [nn.Conv2D(in_channels=in_channels,out_channels=out_channels,kernel_size=3,padding=1),nn.ReLU()]
    for i in range(num_convs-1):
        net.append(nn.Conv2D(out_channels=out_channels,in_channels=out_channels,kernel_size=3,stride=1,padding=1))
        net.append(nn.ReLU())
    net.append(nn.MaxPool2D(kernel_size=2))
    return nn.Sequential(*net)

def vgg_stack(num_convs,channels):
    net = []
    for n,c in zip(num_convs,channels):
        in_c = c[0]
        out_c = c[1]
        net.append(vgg_block(n,in_c,out_c))
    return nn.Sequential(*net)

class VGG(paddle.nn.Layer):
    def __init__(self,vgg_net) -> None:
        super(VGG,self).__init__()
        self.conv = vgg_stack(vgg_net[0],vgg_net[1])
        self.line = nn.Sequential(
            nn.Linear(512*7*7,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,1)
        )
        
    def forward(self,x):
        x = self.conv(x)
        x = paddle.flatten(x, 1, -1)
        x = self.line(x)
        return x

In [7]:
xtemp = paddle.reshape(paddle.to_tensor(mydataset[0][0]),(1,-1,IMAGE_SIZE,IMAGE_SIZE))

In [8]:
xtemp

Tensor(shape=[1, 3, 224, 224], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       [[[[0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          ...,
          [0.21093750, 0.12109375, 0.36718750, ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.21484375,
           0.52734375, 0.35156250],
          [0.20703125, 0.52343750, 0.34765625, ..., 0.19921875,
           0.11328125, 0.37109375]],

         [[0.20703125, 0.11718750, 0.35546875, ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.20703125,
           0.53125000, 0.35546875],
          [0.21484375, 0.51562500, 0.34765625, ..., 0.17578125,
           0.10546875, 0.39062500

In [18]:
net = vgg_block(1,3,2)
net(xtemp)

Tensor(shape=[1, 2, 112, 112], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[[[0.13676271, 0.        , 0.14152804, ..., 0.        ,
           0.        , 0.        ],
          [0.04784188, 0.        , 0.06437561, ..., 0.06340392,
           0.        , 0.10726637],
          [0.        , 0.        , 0.        , ..., 0.07173494,
           0.        , 0.08481816],
          ...,
          [0.07818490, 0.25637057, 0.23291185, ..., 0.39956993,
           0.39525646, 0.25941703],
          [0.23876809, 0.36451370, 0.13031682, ..., 0.02801993,
           0.25859800, 0.26548719],
          [0.29248589, 0.53420007, 0.24558130, ..., 0.37744570,
           0.36665297, 0.22346997]],

         [[0.20554818, 0.08493575, 0.20039070, ..., 0.56644416,
           0.55475503, 0.11925802],
          [0.31906304, 0.53919768, 0.27188921, ..., 0.29144180,
           0.13819334, 0.29434907],
          [0.31809121, 0.50931019, 0.20628865, ..., 0.54256845,
           0.53975648, 0.2573989

In [35]:
net = vgg_stack([2,2,3,3,3], [[3,64],[64,128],[128,256],[256,512],[512,512]])
net(xtemp)

Tensor(shape=[1, 512, 7, 7], dtype=float32, place=Place(gpu:0), stop_gradient=False,
       [[[[0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          ...,
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.00945184, 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ]],

         [[0.        , 0.        , 0.03658355, ..., 0.06424876,
           0.00473395, 0.16345757],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ]

In [5]:
paddle.summary(net=VGG([[2,2,3,3,3], [[3,64],[64,128],[128,256],[256,512],[512,512]]]),input_size=(1,3,224,224))

W0203 11:01:46.058907 74870 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 6.1, Driver API Version: 12.0, Runtime API Version: 11.7
W0203 11:01:46.069173 74870 gpu_resources.cc:91] device: 0, cuDNN Version: 8.7.


---------------------------------------------------------------------------
 Layer (type)       Input Shape          Output Shape         Param #    
   Conv2D-1      [[1, 3, 224, 224]]   [1, 64, 224, 224]        1,792     
    ReLU-1      [[1, 64, 224, 224]]   [1, 64, 224, 224]          0       
   Conv2D-2     [[1, 64, 224, 224]]   [1, 64, 224, 224]       36,928     
    ReLU-2      [[1, 64, 224, 224]]   [1, 64, 224, 224]          0       
  MaxPool2D-1   [[1, 64, 224, 224]]   [1, 64, 112, 112]          0       
   Conv2D-3     [[1, 64, 112, 112]]   [1, 128, 112, 112]      73,856     
    ReLU-3      [[1, 128, 112, 112]]  [1, 128, 112, 112]         0       
   Conv2D-4     [[1, 128, 112, 112]]  [1, 128, 112, 112]      147,584    
    ReLU-4      [[1, 128, 112, 112]]  [1, 128, 112, 112]         0       
  MaxPool2D-2   [[1, 128, 112, 112]]   [1, 128, 56, 56]          0       
   Conv2D-5      [[1, 128, 56, 56]]    [1, 256, 56, 56]       295,168    
    ReLU-5       [[1, 256, 56, 56]] 

{'total_params': 134264641, 'trainable_params': 134264641}

In [11]:
net = VGG([[2,2,3,3,3], [[3,64],[64,128],[128,256],[256,512],[512,512]]])
net(xtemp)

ValueError: mode argument should be 'downscale_in_infer' or 'upscale_in_train'

In [ ]:
line = nn.Sequential(
    nn.Linear(512*7*7,4096),
    nn.ReLU(),
    nn.Dropout(0.5,mode='up_scale_train'),
    nn.Linear(4096,4096),
    nn.ReLU(),
    nn.Dropout(0.5,mode='up_scale_train'),
    nn.Linear(4096,1)
)
line()

: 

: 